# **Importamos la librerias necesarias**

In [191]:
from _utils import *

# **Ahora cargamos los dataset necesarios**

In [192]:
df = pd.DataFrame(read_json("../../datasets/json/coins.json"))

# **Inspeccion de Datos**


Ahora le damos una vista general de los datos y se limpiaran los datos 

`df_coins`

In [193]:
df.head(5)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,62485.000000,1.234314e+12,1.0,1.316328e+12,1.745832e+10,64258.000,...,2024-03-14T07:10:36.635Z,67.810000,9.233951e+04,2013-07-06T00:00:00.000Z,None,2024-04-29T03:07:50.739Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, upbit, ok...",[thorswap]
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,3204.580000,3.933157e+11,2.0,3.933157e+11,1.234581e+10,3345.290,...,2021-11-10T14:24:19.604Z,0.432979,7.444012e+05,2015-10-20T00:00:00.000Z,"{'times': 67.51867213370284, 'currency': 'btc'...",2024-04-29T03:07:34.938Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, upbit, ok...","[curve_ethereum, thorswap, openocean_finance]"
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999644,1.104947e+11,3.0,1.104947e+11,2.619696e+10,1.001,...,2018-07-24T00:00:00.000Z,0.572521,7.468564e+01,2015-03-02T00:00:00.000Z,None,2024-04-29T03:05:41.166Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, okex, huo...","[jupiter, raydium2, curve_ethereum, meteora, t..."
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,592.640000,9.147265e+10,4.0,9.147265e+10,7.763166e+08,606.920,...,2021-05-10T07:24:17.097Z,0.039818,1.493039e+06,2017-10-19T00:00:00.000Z,None,2024-04-29T03:07:39.939Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, okex, huo...","[uniswap_v3, thorswap, openocean_finance]"
4,solana,sol,Solana,https://assets.coingecko.com/coins/images/4128...,135.710000,6.090000e+10,5.0,7.827803e+10,2.231952e+09,144.580,...,2021-11-06T21:54:35.825Z,0.500801,2.710871e+04,2020-05-11T19:35:23.449Z,None,2024-04-29T03:07:43.805Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, upbit, ok...",[]


In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7575 entries, 0 to 7574
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                7575 non-null   object 
 1   symbol                            7575 non-null   object 
 2   name                              7575 non-null   object 
 3   image                             7575 non-null   object 
 4   current_price                     7573 non-null   float64
 5   market_cap                        7574 non-null   float64
 6   market_cap_rank                   3622 non-null   float64
 7   fully_diluted_valuation           6961 non-null   float64
 8   total_volume                      7574 non-null   float64
 9   high_24h                          7321 non-null   float64
 10  low_24h                           7321 non-null   float64
 11  price_change_24h                  7321 non-null   float64
 12  price_

Observamos la frecuencia de las exchanges que pertenece cada moneda

In [195]:
# Cantidad de exchanges centralizados (frecuencia)
df["exchanges_centralized"].apply(lambda x: len(x)).value_counts()[:10]

exchanges_centralized
0    3376
1    2166
2     608
3     316
4     206
5     113
6     110
9      66
7      65
8      58
Name: count, dtype: int64

In [196]:
# Cantidad de exchanges descentralizados (frecuencia)
df["exchanges_decentralized"].apply(lambda x: len(x)).value_counts()[:10]

exchanges_decentralized
1    3193
0    2438
2    1110
3     428
4     225
5     102
6      34
9      11
8      11
7      10
Name: count, dtype: int64

## Caso especial

Antes de realizar la limpieza de los datos, podemos observar los siguiente del `JSON` del dataset:
- La exchange a la cual pertenece es una version de esta pasada `uniswap_v2`, ya que actualmente esta la version 3.
- La criptomoneda no ha tenido movimiento las ultimas 24 horas.
- No tiene maximos ni minimos historicos.
  
Por lo cual podemos deducir que se debe a un error en el registro de la moneda o un problema con la version de la exchange de donde la API nos proporciono esta información 

```json
{
	"id": "party",
	"symbol": "party",
	"name": "PARTY",
	"image": "https://assets.coingecko.com/coins/images/33550/large/party-logo-200x200.png?1702397278",
	"current_price": null,
	"market_cap": null,
	"market_cap_rank": null,
	"fully_diluted_valuation": null,
	"total_volume": null,
	"high_24h": null,
	"low_24h": null,
	"price_change_24h": null,
	"price_change_percentage_24h": null,
	"market_cap_change_24h": null,
	"market_cap_change_percentage_24h": null,
	"circulating_supply": null,
	"total_supply": 420177600000000.0,
	"max_supply": 420177600000000.0,
	"ath": null,
	"ath_change_percentage": 0.0,
	"ath_date": null,
	"atl": null,
	"atl_change_percentage": 0.0,
	"atl_date": null,
	"roi": null,
	"last_updated": null,
	"category": "Pixels Game",
	"category_id": "pixels-game",
	"exchanges_centralized": [],
	"exchanges_decentralized": [
		"uniswap_v2"
	]
}
```

Este caso especial se tendra que eliminar del dataset 

In [197]:
# Eliminar la fila con id "party"
df = df[df["id"] != "party"]

In [198]:
# Validamos que no existe ese json
df[df["id"] == "party"]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized


# **Preprocesamiento de los Datos**

Eliminamos primero la columna `roi`, ya que no nos proporciona informacion relevante para el analisis

In [199]:
# Eliminamos la columna `roi`
df.drop(columns=["roi"], inplace=True)

Ahora verificamos que columnas tienen valores nulos

In [200]:
find_columns_with_nulls(df)

[('current_price', 1),
 ('market_cap_rank', 3952),
 ('fully_diluted_valuation', 613),
 ('high_24h', 253),
 ('low_24h', 253),
 ('price_change_24h', 253),
 ('price_change_percentage_24h', 253),
 ('market_cap_change_24h', 255),
 ('market_cap_change_percentage_24h', 255),
 ('total_supply', 505),
 ('max_supply', 2278)]

Eliminamos el unico registro que no tiene `current_price`

In [201]:
df = df[~df["current_price"].isnull()]

## Ahora revisamos la columna de `market_cap_rank`

In [202]:
find_columns_with_nulls(df)[0]

('market_cap_rank', 3951)

Observamos el maximo valor de `market_cap_rank`

In [203]:
df["market_cap_rank"].max() 

4333.0

In [204]:
df[3622:].head(4)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized
3622,mainstream-for-the-underground,mftu,Mainstream For The Underground,https://assets.coingecko.com/coins/images/5519...,1.000000e-17,7.771000e-09,4333.0,1.379400e-08,0.00,1.000000e-17,...,-100.00000,2020-10-06T16:57:17.044Z,1.000000e-17,0.41575,2023-04-20T20:00:10.295Z,2024-04-29T00:32:38.909Z,Pixels Game,pixels-game,[latoken],[]
3623,kryxivia-game,kxa,Kryxivia Game,https://assets.coingecko.com/coins/images/2122...,2.094190e-03,0.000000e+00,NaN,5.235490e+05,1685.38,2.198170e-03,...,-98.87854,2021-12-05T23:31:02.116Z,2.003640e-03,4.51938,2024-04-27T16:50:10.553Z,2024-04-29T03:02:31.532Z,Adventure Games,adventure-games,[],[uniswap_v3]
3624,0xdefcafe,cafe,0xDEFCAFE,https://assets.coingecko.com/coins/images/3173...,1.068499e-02,0.000000e+00,NaN,1.068850e+05,226.15,1.085556e-02,...,-63.82070,2023-10-31T16:50:59.461Z,1.036232e-02,3.37069,2024-04-14T03:15:56.230Z,2024-04-29T02:56:56.671Z,Artificial Intelligence (AI),artificial-intelligence,[],[uniswap_v2]
3625,architex,arcx,Architex,https://assets.coingecko.com/coins/images/3602...,1.510000e+00,0.000000e+00,NaN,1.510424e+07,258080.00,1.730000e+00,...,-87.21325,2024-03-28T10:45:47.363Z,1.280000e+00,18.13056,2024-04-26T19:34:11.029Z,2024-04-29T02:57:43.500Z,Artificial Intelligence (AI),artificial-intelligence,[poloniex],[]


Observamos que solo hay ranking hasta el numero `4333`, asi que rellenamos los nulos de esta columna con un `-1`

In [205]:
df["market_cap_rank"].fillna(-1, inplace=True) # Rellenar con -1
df["market_cap_rank"] = df["market_cap_rank"].astype(int) # Convertir a entero

C:\Users\pms_l\AppData\Local\Temp\ipykernel_28440\712580403.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["market_cap_rank"].fillna(-1, inplace=True) # Rellenar con -1


## Ahora revisamos la columna `fully_diluted_valuation`

In [206]:
find_columns_with_nulls(df)[0]

('fully_diluted_valuation', 612)

Observamos a detalle y notamos que:
- `fully_diluted_valuation` es igual a `total_supply * current_price` en ciertos casos
- Cuando `total_supply` es nulo, `fully_diluted_valuation` tambien es nulo


In [207]:
# Monedas con valor de `fully_diluted_valuation` igual al valor de `total_supply` por `current_price`
df[df["fully_diluted_valuation"] == df["total_supply"]*df["current_price"]].iloc[:3, 4:20]

,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
2787,0.799023,1185137.0,2597,1598046.0,72.09,0.824169,0.799023,-0.022018,-2.68171,-32901.496704,-2.70119,1.483233e+06,2000000.0,2000000.0,12.480000,-93.59984
2854,0.005253,1008322.0,2690,2626410.0,1.80,0.005269,0.005251,-0.000013,-0.25395,-6659.805791,-0.65615,1.919583e+08,500000000.0,500000000.0,0.058478,-91.01749
2855,0.005253,1008322.0,2690,2626410.0,1.80,0.005269,0.005251,-0.000013,-0.25395,-6659.805791,-0.65615,1.919583e+08,500000000.0,500000000.0,0.058478,-91.01749


In [208]:
df[df["fully_diluted_valuation"].isnull()].iloc[:3, 7:20]

,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
64,NaN,53629654.0,129.630000,123.130000,1.450000,1.16776,5.687159e+07,2.54063,1.814782e+07,NaN,NaN,542.330,-76.68940
136,NaN,95653952.0,0.825745,0.786902,-0.037229,-4.52423,-3.290268e+07,-3.49946,1.148373e+09,NaN,NaN,22.710,-96.52354
160,NaN,4729970.0,0.984959,0.964619,-0.009433,-0.96082,-8.910602e+06,-1.24946,7.253320e+08,NaN,NaN,1.052,-7.35901


Entonces rellenaremos con el valor de `total_supply * current_price` cuando `fully_diluted_valuation` sea nulo y `total_supply` no sea nulo

In [209]:
df["fully_diluted_valuation"].fillna(df["total_supply"]*df["current_price"], inplace=True)

C:\Users\pms_l\AppData\Local\Temp\ipykernel_28440\1657751060.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["fully_diluted_valuation"].fillna(df["total_supply"]*df["current_price"], inplace=True)


In [210]:
find_columns_with_nulls(df)[0]

('fully_diluted_valuation', 504)

Ahora los restantes que no tenemos datos rellenaremos con -1

In [211]:
df["fully_diluted_valuation"].fillna(-1, inplace=True) # Rellenar con -1

C:\Users\pms_l\AppData\Local\Temp\ipykernel_28440\3207985362.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["fully_diluted_valuation"].fillna(-1, inplace=True) # Rellenar con -1


## Ahora verificamos la columna `high_24h`


In [212]:
find_columns_with_nulls(df)[0]

('high_24h', 252)

In [213]:
df[df["high_24h"].isnull()].iloc[:5, 0:20]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
903,ecoin-2,ecoin,Ecoin,https://assets.coingecko.com/coins/images/1084...,1.106400e-04,51020629.0,681,110644598.0,1.80,NaN,NaN,NaN,NaN,NaN,NaN,4.611217e+11,1.000000e+12,NaN,0.012765,-99.13320
1469,ethereum-meta,ethm,Ethereum Meta,https://assets.coingecko.com/coins/images/6586...,1.736100e-11,17378399.0,1165,17378399.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,1.000991e+18,1.000991e+18,NaN,0.041412,-100.00000
2020,earnbet,ebet,EarnBet,https://assets.coingecko.com/coins/images/3203...,1.363090e-03,6371987.0,1674,9961048.0,1463.43,NaN,NaN,NaN,NaN,NaN,NaN,4.674656e+09,7.307684e+09,8.800000e+09,0.001761,-22.60582
2092,nix-bridge-token,voice,Voice,https://assets.coingecko.com/coins/images/1292...,1.613000e+02,5667067.0,1744,7195831.0,1290.38,NaN,NaN,NaN,NaN,NaN,NaN,3.513412e+04,4.461200e+04,NaN,562.520000,-71.32559
2106,molecules-of-korolchuk-ip-nft,vita-fast,Molecules of Korolchuk IP-NFT,https://assets.coingecko.com/coins/images/3305...,5.500000e+00,5500752.0,1757,5500752.0,150.37,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+06,1.000000e+06,1.000000e+06,16.210000,-66.06971


Observamos que en otros registros el high_24h es igual al current_price, por lo cual rellenaremos los nulos con el valor de `current_price`

In [214]:
df[df["current_price"] == df["high_24h"]][:5]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized
1530,vector-reserve,vec,Vector Reserve,https://assets.coingecko.com/coins/images/3487...,1.193000e+01,15915217.0,1216,46933434.0,342542.00,1.193000e+01,...,-79.82840,2024-02-05T20:04:43.769Z,8.800000e+00,33.04180,2024-04-28T00:14:56.974Z,2024-04-29T03:07:22.191Z,Pixels Game,pixels-game,[poloniex],[uniswap_v2]
2295,holdstation-usd-coin,hsusdc,Holdstation USDC,https://assets.coingecko.com/coins/images/3264...,1.130000e+00,3460462.0,1983,-1.0,3619.55,1.130000e+00,...,-2.57158,2024-03-18T01:32:29.495Z,9.205730e-01,22.55011,2024-02-29T00:27:55.670Z,2024-04-29T03:02:42.259Z,Pixels Game,pixels-game,[],[syncswap]
2829,lunar,lnr,Lunar [OLD],https://assets.coingecko.com/coins/images/2044...,1.353000e-09,1085277.0,2655,1238214.0,0.00,1.353000e-09,...,-97.36068,2021-11-30T11:08:46.028Z,1.273000e-09,6.29404,2022-10-21T08:45:46.552Z,2024-04-29T03:08:21.476Z,Pixels Game,pixels-game,[digifinex],[]
2835,dentacoin,dcn,Dentacoin,https://assets.coingecko.com/coins/images/850/...,1.480000e-06,1054339.0,2665,11865851.0,6.99,1.480000e-06,...,-99.97522,2018-01-07T00:00:00.000Z,4.023500e-08,3586.07143,2021-01-27T19:51:03.335Z,2024-04-29T03:07:51.650Z,Pixels Game,pixels-game,[latoken],[]
3038,quiverx,qrx,QuiverX,https://assets.coingecko.com/coins/images/1291...,6.763960e-03,616187.0,2974,676396.0,243.43,6.763960e-03,...,-95.59579,2021-03-24T12:47:16.747Z,1.402340e-03,382.33463,2022-10-20T14:31:30.132Z,2024-04-28T15:42:29.563Z,Pixels Game,pixels-game,[],[uniswap_v2]


In [215]:
df["high_24h"].fillna(df["current_price"], inplace=True)

C:\Users\pms_l\AppData\Local\Temp\ipykernel_28440\3666677201.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["high_24h"].fillna(df["current_price"], inplace=True)


## Verificamos la columna `low_24h`

In [216]:
find_columns_with_nulls(df)[0]

('low_24h', 252)

In [217]:
df[df["low_24h"].isnull()].iloc[:5, 0:20]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
903,ecoin-2,ecoin,Ecoin,https://assets.coingecko.com/coins/images/1084...,1.106400e-04,51020629.0,681,110644598.0,1.80,1.106400e-04,NaN,NaN,NaN,NaN,NaN,4.611217e+11,1.000000e+12,NaN,0.012765,-99.13320
1469,ethereum-meta,ethm,Ethereum Meta,https://assets.coingecko.com/coins/images/6586...,1.736100e-11,17378399.0,1165,17378399.0,0.00,1.736100e-11,NaN,NaN,NaN,NaN,NaN,1.000991e+18,1.000991e+18,NaN,0.041412,-100.00000
2020,earnbet,ebet,EarnBet,https://assets.coingecko.com/coins/images/3203...,1.363090e-03,6371987.0,1674,9961048.0,1463.43,1.363090e-03,NaN,NaN,NaN,NaN,NaN,4.674656e+09,7.307684e+09,8.800000e+09,0.001761,-22.60582
2092,nix-bridge-token,voice,Voice,https://assets.coingecko.com/coins/images/1292...,1.613000e+02,5667067.0,1744,7195831.0,1290.38,1.613000e+02,NaN,NaN,NaN,NaN,NaN,3.513412e+04,4.461200e+04,NaN,562.520000,-71.32559
2106,molecules-of-korolchuk-ip-nft,vita-fast,Molecules of Korolchuk IP-NFT,https://assets.coingecko.com/coins/images/3305...,5.500000e+00,5500752.0,1757,5500752.0,150.37,5.500000e+00,NaN,NaN,NaN,NaN,NaN,1.000000e+06,1.000000e+06,1.000000e+06,16.210000,-66.06971


Observamos que en otros registros el low_24h es igual al current_price, por lo cual rellenaremos los nulos con el valor de `current_price`


In [218]:
df[df["current_price"] == df["low_24h"]][:5]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized
26,leo-token,leo,LEO Token,https://assets.coingecko.com/coins/images/8418...,5.73,5.315659e+09,24,5.652468e+09,1154916.0,5.87,...,-29.42999,2022-02-08T17:40:10.285Z,0.799859,617.76547,2019-12-24T15:14:35.376Z,2024-04-29T03:07:38.281Z,Pixels Game,pixels-game,"[okex, digifinex, hotcoin_global, lbank, xt, p...",[uniswap_v3]
32,blockstack,stx,Stacks,https://assets.coingecko.com/coins/images/2069...,2.43,3.553893e+09,30,4.441532e+09,44032025.0,2.60,...,-36.77839,2024-04-01T12:34:58.342Z,0.045596,5258.07039,2020-03-13T02:29:26.415Z,2024-04-29T03:07:29.082Z,Pixels Game,pixels-game,"[binance, bybit_spot, upbit, okex, bitget, bit...",[]
39,okb,okb,OKB,https://assets.coingecko.com/coins/images/4463...,51.60,3.103634e+09,37,1.220544e+10,6166786.0,53.63,...,-30.06146,2024-03-14T00:30:12.502Z,0.580608,8790.16052,2019-01-14T00:00:00.000Z,2024-04-29T03:07:36.101Z,Pixels Game,pixels-game,"[okex, bitrue, hotcoin_global, bitmart, lbank,...",[uniswap_v3]
44,immutable-x,imx,Immutable,https://assets.coingecko.com/coins/images/1723...,2.08,3.041103e+09,40,4.175001e+09,43997710.0,2.19,...,-78.09319,2021-11-26T01:03:01.536Z,0.378055,451.64342,2022-12-31T07:36:37.649Z,2024-04-29T03:06:54.971Z,Gaming (GameFi),gaming,"[binance, bybit_spot, upbit, okex, huobi, bitg...",[]
45,immutable-x,imx,Immutable,https://assets.coingecko.com/coins/images/1723...,2.08,3.041103e+09,40,4.175001e+09,44103598.0,2.19,...,-78.09319,2021-11-26T01:03:01.536Z,0.378055,451.64342,2022-12-31T07:36:37.649Z,2024-04-29T03:07:27.779Z,Pixels Game,pixels-game,"[binance, bybit_spot, upbit, okex, huobi, bitg...",[]


In [219]:
df["low_24h"].fillna(df["current_price"], inplace=True)

C:\Users\pms_l\AppData\Local\Temp\ipykernel_28440\2957584048.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["low_24h"].fillna(df["current_price"], inplace=True)


## Verificamos la columna `price_change_24h` y `price_change_percentage_24h`

In [220]:
find_columns_with_nulls(df)[0]

('price_change_24h', 252)

In [221]:
df[df["price_change_24h"].isnull()].iloc[:5, 0:20]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
903,ecoin-2,ecoin,Ecoin,https://assets.coingecko.com/coins/images/1084...,1.106400e-04,51020629.0,681,110644598.0,1.80,1.106400e-04,1.106400e-04,NaN,NaN,NaN,NaN,4.611217e+11,1.000000e+12,NaN,0.012765,-99.13320
1469,ethereum-meta,ethm,Ethereum Meta,https://assets.coingecko.com/coins/images/6586...,1.736100e-11,17378399.0,1165,17378399.0,0.00,1.736100e-11,1.736100e-11,NaN,NaN,NaN,NaN,1.000991e+18,1.000991e+18,NaN,0.041412,-100.00000
2020,earnbet,ebet,EarnBet,https://assets.coingecko.com/coins/images/3203...,1.363090e-03,6371987.0,1674,9961048.0,1463.43,1.363090e-03,1.363090e-03,NaN,NaN,NaN,NaN,4.674656e+09,7.307684e+09,8.800000e+09,0.001761,-22.60582
2092,nix-bridge-token,voice,Voice,https://assets.coingecko.com/coins/images/1292...,1.613000e+02,5667067.0,1744,7195831.0,1290.38,1.613000e+02,1.613000e+02,NaN,NaN,NaN,NaN,3.513412e+04,4.461200e+04,NaN,562.520000,-71.32559
2106,molecules-of-korolchuk-ip-nft,vita-fast,Molecules of Korolchuk IP-NFT,https://assets.coingecko.com/coins/images/3305...,5.500000e+00,5500752.0,1757,5500752.0,150.37,5.500000e+00,5.500000e+00,NaN,NaN,NaN,NaN,1.000000e+06,1.000000e+06,1.000000e+06,16.210000,-66.06971


In [222]:
df["price_change_24h"].fillna(0, inplace=True)
df["price_change_percentage_24h"].fillna(0, inplace=True)

C:\Users\pms_l\AppData\Local\Temp\ipykernel_28440\1736215031.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["price_change_24h"].fillna(0, inplace=True)
C:\Users\pms_l\AppData\Local\Temp\ipykernel_28440\1736215031.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

## Verificamos la columna `market_cap_change_24h` y `market_cap_change_percentage_24h`


In [223]:
find_columns_with_nulls(df)[0]

('market_cap_change_24h', 254)

In [224]:
find_columns_with_nulls(df)[1]

('market_cap_change_percentage_24h', 254)

In [225]:
df[df["market_cap_change_24h"].isnull()].iloc[:5, 5:20]

,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
903,51020629.0,681,110644598.0,1.80,1.106400e-04,1.106400e-04,0.000000,0.00000,NaN,NaN,4.611217e+11,1.000000e+12,NaN,0.012765,-99.13320
1325,22756674.0,1043,24736471.0,434.63,7.706900e-02,7.510700e-02,-0.001713,-2.22426,NaN,NaN,3.022678e+08,3.285647e+08,NaN,0.923577,-91.84838
1469,17378399.0,1165,17378399.0,0.00,1.736100e-11,1.736100e-11,0.000000,0.00000,NaN,NaN,1.000991e+18,1.000991e+18,NaN,0.041412,-100.00000
1688,11673014.0,1358,11673014.0,39490.00,1.214382e+04,8.100340e+03,3557.000000,41.92895,NaN,NaN,9.694870e+02,9.694870e+02,9.694870e+02,166135.000000,-92.75263
2020,6371987.0,1674,9961048.0,1463.43,1.363090e-03,1.363090e-03,0.000000,0.00000,NaN,NaN,4.674656e+09,7.307684e+09,8.800000e+09,0.001761,-22.60582


In [ ]:
df["market_cap_change_24h"].fillna(0, inplace=True)
df["market_cap_change_percentage_24h"].fillna(0, inplace=True)

## Verificamos la columna `total_supply`


In [227]:
find_columns_with_nulls(df)[0]

('total_supply', 504)

In [228]:
df[df["total_supply"].isnull()].iloc[:5, 15:20]

,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
64,1.814782e+07,NaN,NaN,542.330000,-76.68940
136,1.148373e+09,NaN,NaN,22.710000,-96.52354
160,7.253320e+08,NaN,NaN,1.052000,-7.35901
329,7.544039e+10,NaN,NaN,0.046712,-92.68592
350,1.448188e+08,NaN,NaN,34.920000,-95.29145


Supondremos que el `circulating_supply` es el mismo que `total_supply`, ya que en algunas monedas es comun esto

In [229]:
df[df["circulating_supply"] != df["total_supply"]].iloc[:5, 15:20]

,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
0,1.969159e+07,2.100000e+07,2.100000e+07,73738.000000,-14.99314
4,4.472827e+08,5.749164e+08,NaN,259.960000,-47.58350
5,3.349472e+10,3.348919e+10,NaN,1.170000,-14.76402
7,5.523591e+10,9.998765e+10,1.000000e+11,3.400000,-85.02530
8,1.440775e+11,1.440784e+11,NaN,0.731578,-80.15866


In [ ]:
df["total_supply"].fillna(df["circulating_supply"], inplace=True)

## Verificamos la columna `max_supply`

In [231]:
find_columns_with_nulls(df)[0]

('max_supply', 2277)

In [232]:
df[df["max_supply"].isnull()].iloc[:5, 15:20]

,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
1,1.220549e+08,1.220549e+08,NaN,4878.26,-33.92044
2,1.105629e+11,1.105629e+11,NaN,1.32,-24.41119
4,4.472827e+08,5.749164e+08,NaN,259.96,-47.58350
5,3.349472e+10,3.348919e+10,NaN,1.17,-14.76402
6,9.345421e+06,9.345421e+06,NaN,4829.57,-33.27459


Que `max_supply` sa nulo puede significar que la moneda no tiene limite de emision

In [ ]:
df["max_supply"].fillna(-1, inplace=True)

Una vez limpio los datos podemos observar los datos estadisticos de estos

In [234]:
df.describe()

,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,atl,atl_change_percentage
count,7573.000000,7.573000e+03,7573.000000,7.573000e+03,7.573000e+03,7573.000000,7573.000000,7.573000e+03,7573.000000,7.573000e+03,7573.000000,7.573000e+03,7.573000e+03,7.573000e+03,7.573000e+03,7573.000000,7573.000000,7.573000e+03
mean,176.394779,3.426023e+08,793.403803,2.238419e+21,1.303426e+07,182.901268,174.478980,-3.666254e+00,-1.505581,-8.095198e+06,0.146703,8.460250e+19,1.320763e+25,1.440567e+21,7.043349e+03,-78.925092,54.668234,2.510893e+42
std,2979.733860,1.501799e+10,1145.477602,1.947939e+23,3.954865e+08,3084.134047,2948.601762,9.175849e+01,19.821238,3.097437e+08,40.991559,5.404619e+21,1.149122e+27,1.151160e+23,3.729459e+05,25.229414,1305.423492,2.129564e+44
min,0.000000,0.000000e+00,-1.000000,-1.000000e+00,0.000000e+00,0.000000,0.000000,-2.779702e+03,-96.430990,-2.434849e+10,-89.596320,0.000000e+00,0.000000e+00,-2.000000e+00,0.000000e+00,-100.000000,-0.009466,-1.311994e+02
25%,0.000364,0.000000e+00,-1.000000,3.073700e+05,1.865120e+03,0.000398,0.000344,-2.354612e-03,-4.906870,-4.697212e+04,-1.604870,0.000000e+00,6.449348e+07,-1.000000e+00,7.229660e-03,-98.003130,0.000077,3.302201e+01
50%,0.009530,0.000000e+00,-1.000000,3.428523e+06,5.161800e+04,0.010515,0.009114,-2.361118e-05,-1.975340,0.000000e+00,0.000000,0.000000e+00,8.000000e+08,1.188256e+08,2.131870e-01,-90.027020,0.002305,1.425420e+02
75%,0.182146,8.058333e+06,1394.000000,3.472978e+07,4.784780e+05,0.191035,0.177861,4.795000e-09,0.130820,0.000000e+00,0.000000,2.373917e+08,4.746558e+09,1.000000e+09,2.260000e+00,-67.625630,0.040719,6.129274e+02
max,81188.000000,1.234314e+12,4333.000000,1.695155e+25,2.619696e+10,84499.000000,76842.000000,3.557000e+03,1208.182180,1.541906e+08,2325.617430,4.507743e+23,1.000000e+29,1.000000e+25,3.092736e+07,0.006780,67341.000000,1.852571e+46


Guardamos el dataset limpio


In [239]:
df.to_csv("../../datasets/csv/coins_clean.csv", index=False)

In [237]:
df.to_json("../../datasets/json/coins_clean.json", orient="records")